Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in `1_notmnist.ipynb`.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [20]:
batch_size = 128
lam = 0.01
graph_1 = tf.Graph_1()
with graph_1.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))+\
    lam*tf.nn.l2_loss(weights)
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [21]:
%%timeit -n 1 -r 1
num_steps = 3001

with tf.Session(graph=graph_1) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 46.977703
Minibatch accuracy: 10.9%
Validation accuracy: 10.3%
Minibatch loss at step 500: 0.969443
Minibatch accuracy: 77.3%
Validation accuracy: 81.2%
Minibatch loss at step 1000: 0.743931
Minibatch accuracy: 82.0%
Validation accuracy: 82.2%
Minibatch loss at step 1500: 0.688336
Minibatch accuracy: 83.6%
Validation accuracy: 81.7%
Minibatch loss at step 2000: 0.606634
Minibatch accuracy: 85.2%
Validation accuracy: 81.2%
Minibatch loss at step 2500: 0.949985
Minibatch accuracy: 75.0%
Validation accuracy: 80.1%
Minibatch loss at step 3000: 0.822875
Minibatch accuracy: 82.8%
Validation accuracy: 82.1%
Test accuracy: 88.4%
1 loop, best of 1: 5.3 s per loop


In [17]:
batch_size = 128
lam = 0.01
graph_2 = tf.Graph()
with graph_2.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  num_hidden = 1024  
  #layer 1
  W1 = tf.Variable(tf.truncated_normal([image_size * image_size, num_hidden]))
  b1 = tf.Variable(tf.zeros([num_hidden]))


  #layer 2
  W2 = tf.Variable(tf.truncated_normal([num_hidden, num_labels]))
  b2 = tf.Variable(tf.zeros([num_labels]))

  
  # Training computation.
  # hidden layer output  
  y1 = tf.nn.relu(tf.matmul(tf_train_dataset, W1) + b1)
  # second layer output
  y2 = tf.matmul(y1, W2) + b2
  ## QUESTION : IS THIS THE RIGHT WAY TO REGULARIZE?
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=y2))+\
    lam*(tf.nn.l2_loss(W1)+tf.nn.l2_loss(W2)) 
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(y2)
  y1_valid = tf.nn.relu(tf.matmul(tf_valid_dataset, W1) + b1)  
  valid_prediction = tf.nn.softmax(tf.matmul(y1_valid, W2) + b2)
  y1_test = tf.nn.relu(tf.matmul(tf_test_dataset, W1) + b1)  
  test_prediction = tf.nn.softmax(tf.matmul(y1_test, W2) + b2)

In [37]:
%%timeit -n 1 -r 1
num_steps = 3001

with tf.Session(graph=graph_2) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3457.539307
Minibatch accuracy: 6.2%
Validation accuracy: 23.6%
Minibatch loss at step 500: 21.342869
Minibatch accuracy: 82.0%
Validation accuracy: 84.9%
Minibatch loss at step 1000: 0.855735
Minibatch accuracy: 82.8%
Validation accuracy: 83.6%
Minibatch loss at step 1500: 0.684081
Minibatch accuracy: 84.4%
Validation accuracy: 84.2%
Minibatch loss at step 2000: 0.653804
Minibatch accuracy: 85.9%
Validation accuracy: 84.0%
Minibatch loss at step 2500: 0.888672
Minibatch accuracy: 79.7%
Validation accuracy: 83.0%
Minibatch loss at step 3000: 0.774730
Minibatch accuracy: 85.2%
Validation accuracy: 84.5%
Test accuracy: 90.7%
1 loop, best of 1: 2min 24s per loop


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [14]:
def train_nn(n=None, graph=None):
    num_steps = 3001
    if n:
        train_dataset_small=train_dataset[:n*batch_size]
        train_labels_small=train_labels[:n*batch_size]
    else:
        train_dataset_small=train_dataset
        train_labels_small=train_labels
    
    with tf.Session(graph=graph) as session:
      tf.global_variables_initializer().run()
      print("Initialized")
      for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels_small.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset_small[offset:(offset + batch_size), :]
        batch_labels = train_labels_small[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
          print("Minibatch loss at step %d: %f" % (step, l))
          print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
          print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
      print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))
    

In [21]:
# NOTE : We use the graph defined above, that is, two layer NN with l2 regularization
train_nn(n=10,graph=graph_2)

Initialized
Minibatch loss at step 0: 3507.186523
Minibatch accuracy: 10.2%
Validation accuracy: 28.1%
Minibatch loss at step 500: 21.194187
Minibatch accuracy: 100.0%
Validation accuracy: 80.3%
Minibatch loss at step 1000: 0.602792
Minibatch accuracy: 100.0%
Validation accuracy: 80.7%
Minibatch loss at step 1500: 0.429886
Minibatch accuracy: 100.0%
Validation accuracy: 80.7%
Minibatch loss at step 2000: 0.402515
Minibatch accuracy: 99.2%
Validation accuracy: 80.7%
Minibatch loss at step 2500: 0.393627
Minibatch accuracy: 100.0%
Validation accuracy: 80.5%
Minibatch loss at step 3000: 0.391874
Minibatch accuracy: 100.0%
Validation accuracy: 80.2%
Test accuracy: 87.1%
1 loop, best of 1: 2min 22s per loop


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [31]:
# Let's define a new network : 
# graph_3 = graph with 2 layers, regularization and dropout on the hidden layer
batch_size = 128
lam = 0.01
graph_3 = tf.Graph()
with graph_3.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  num_hidden = 1024  
  #layer 1
  W1 = tf.Variable(tf.truncated_normal([image_size * image_size, num_hidden]))
  b1 = tf.Variable(tf.zeros([num_hidden]))


  #layer 2
  W2 = tf.Variable(tf.truncated_normal([num_hidden, num_labels]))
  b2 = tf.Variable(tf.zeros([num_labels]))

  
  # Training computation.
  # hidden layer output: dropout(relu(xw1+b1))  
  y1 = tf.nn.dropout(tf.nn.relu(tf.matmul(tf_train_dataset, W1) + b1),keep_prob = 0.5)
  # second layer output
  y2 = tf.matmul(y1, W2) + b2
  ## QUESTION : IS THIS THE RIGHT WAY TO REGULARIZE?
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=y2))+\
    lam*(tf.nn.l2_loss(W1)+tf.nn.l2_loss(W2)) 
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(y2)
  # Here we don't put any dropout!!
  y1_valid = tf.nn.relu(tf.matmul(tf_valid_dataset, W1) + b1)  
  valid_prediction = tf.nn.softmax(tf.matmul(y1_valid, W2) + b2)
  y1_test = tf.nn.relu(tf.matmul(tf_test_dataset, W1) + b1)  
  test_prediction = tf.nn.softmax(tf.matmul(y1_test, W2) + b2)

In [29]:
# Now run it
train_nn(graph=graph_3)

Initialized
Minibatch loss at step 0: 3501.294189
Minibatch accuracy: 10.2%
Validation accuracy: 26.4%
Minibatch loss at step 500: 21.283237
Minibatch accuracy: 84.4%
Validation accuracy: 84.3%
Minibatch loss at step 1000: 0.875911
Minibatch accuracy: 84.4%
Validation accuracy: 83.8%
Minibatch loss at step 1500: 0.700808
Minibatch accuracy: 84.4%
Validation accuracy: 84.3%
Minibatch loss at step 2000: 0.680126
Minibatch accuracy: 86.7%
Validation accuracy: 84.0%
Minibatch loss at step 2500: 0.881668
Minibatch accuracy: 82.8%
Validation accuracy: 83.1%
Minibatch loss at step 3000: 0.800948
Minibatch accuracy: 82.8%
Validation accuracy: 84.6%
Test accuracy: 90.5%


In [32]:
train_nn(n=10,graph=graph_3)

Initialized
Minibatch loss at step 0: 3544.938477
Minibatch accuracy: 14.8%
Validation accuracy: 34.0%
Minibatch loss at step 500: 21.239218
Minibatch accuracy: 100.0%
Validation accuracy: 80.9%
Minibatch loss at step 1000: 0.669482
Minibatch accuracy: 98.4%
Validation accuracy: 80.7%
Minibatch loss at step 1500: 0.487828
Minibatch accuracy: 98.4%
Validation accuracy: 80.8%
Minibatch loss at step 2000: 0.448609
Minibatch accuracy: 98.4%
Validation accuracy: 81.0%
Minibatch loss at step 2500: 0.433872
Minibatch accuracy: 99.2%
Validation accuracy: 80.9%
Minibatch loss at step 3000: 0.442864
Minibatch accuracy: 98.4%
Validation accuracy: 80.3%
Test accuracy: 87.3%


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [40]:
# Let's define a new network : 
# graph_4 = multi layers with decaying learning-rate
batch_size = 128
lam = 0.001
# Variables.
num_hidden_1 = 1024  
num_hidden_2 = 256
num_hidden_3 = 64
    
graph_4 = tf.Graph()
with graph_4.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  

  #layer 1
  W1 = tf.Variable(tf.truncated_normal([image_size * image_size, num_hidden_1],stddev = np.sqrt(2.0/(image_size*image_size))))
  b1 = tf.Variable(tf.zeros([num_hidden_1]))

  #layer 2
  W2 = tf.Variable(tf.truncated_normal([num_hidden_1, num_hidden_2],stddev = np.sqrt(2.0/num_hidden_1)))
  b2 = tf.Variable(tf.zeros([num_hidden_2]))

  #layer 3
  W3 = tf.Variable(tf.truncated_normal([num_hidden_2, num_hidden_3],stddev = np.sqrt(2.0/num_hidden_2)))
  b3 = tf.Variable(tf.zeros([num_hidden_3]))

  #layer 4
  W4 = tf.Variable(tf.truncated_normal([num_hidden_3, num_labels],stddev = np.sqrt(2.0/num_hidden_3)))
  b4 = tf.Variable(tf.zeros([num_labels]))
    
  # Training computation.
  # hidden layer output: dropout(relu(xw1+b1))  
  y1 = tf.nn.dropout(tf.nn.relu(tf.matmul(tf_train_dataset, W1) + b1),keep_prob=0.75)
  # second layer output
  y2 = tf.nn.dropout(tf.nn.relu(tf.matmul(y1, W2) + b2),keep_prob=0.75)
  y3 = tf.nn.dropout(tf.nn.relu(tf.matmul(y2, W3) + b3),keep_prob=0.75)
  y4 = tf.matmul(y3, W4) + b4
  ## QUESTION : IS THIS THE RIGHT WAY TO REGULARIZE?
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=y4))+\
    lam*(tf.nn.l2_loss(W1)+tf.nn.l2_loss(W2)+tf.nn.l2_loss(W3)+tf.nn.l2_loss(W4)) 
    
  # Optimizer.
  global_step = tf.Variable(0)  # count the number of steps taken.
  #learning_rate = tf.train.inverse_time_decay(0.0001, global_step, 1,1)
  learning_rate = tf.train.exponential_decay(0.5, global_step, 1000,0.65,staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(y4)
  # Here we don't put any dropout!!
  y1_valid = tf.nn.relu(tf.matmul(tf_valid_dataset, W1) + b1)  
  y2_valid = tf.nn.relu(tf.matmul(y1_valid, W2) + b2)
  y3_valid = tf.nn.relu(tf.matmul(y2_valid, W3) + b3)
  valid_prediction = tf.nn.softmax(tf.matmul(y3_valid, W4) + b4)
  y1_test = tf.nn.relu(tf.matmul(tf_test_dataset, W1) + b1)  
  y2_test = tf.nn.relu(tf.matmul(y1_test, W2) + b2)
  y3_test = tf.nn.relu(tf.matmul(y2_test, W3) + b3)
  test_prediction = tf.nn.softmax(tf.matmul(y3_test, W4) + b4)

In [41]:
num_steps = 9001
with tf.Session(graph=graph_4) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.406360
Minibatch accuracy: 10.9%
Validation accuracy: 21.8%
Minibatch loss at step 500: 1.260577
Minibatch accuracy: 80.5%
Validation accuracy: 85.4%
Minibatch loss at step 1000: 0.947593
Minibatch accuracy: 82.0%
Validation accuracy: 86.4%
Minibatch loss at step 1500: 0.767547
Minibatch accuracy: 87.5%
Validation accuracy: 87.7%
Minibatch loss at step 2000: 0.605866
Minibatch accuracy: 91.4%
Validation accuracy: 88.0%
Minibatch loss at step 2500: 0.802908
Minibatch accuracy: 83.6%
Validation accuracy: 88.6%
Minibatch loss at step 3000: 0.595342
Minibatch accuracy: 87.5%
Validation accuracy: 88.7%
Minibatch loss at step 3500: 0.475744
Minibatch accuracy: 91.4%
Validation accuracy: 89.2%
Minibatch loss at step 4000: 0.604232
Minibatch accuracy: 87.5%
Validation accuracy: 89.2%
Minibatch loss at step 4500: 0.486223
Minibatch accuracy: 90.6%
Validation accuracy: 89.6%
Minibatch loss at step 5000: 0.639734
Minibatch accuracy: 87.5%
Validation accurac